In [1]:
!cp -r /kaggle/input/col774a3/* /kaggle/working/


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.optim.lr_scheduler import StepLR
import os
import pickle
import pandas as pd
from torch.utils.data import DataLoader, Dataset

# Set a confidence threshold (will be dynamically generated for each class)
PENALTY_WEIGHT = 10  # Weight for penalizing incorrect predictions after 50% accuracy
SAVE_PATH = './saved_models/'  # Directory to save model checkpoints
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)

# Temperature scaling class
class TemperatureScaling(nn.Module):
    def __init__(self, init_temp=1.5):
        super(TemperatureScaling, self).__init__()
        self.temperature = nn.Parameter(torch.ones(1) * init_temp)

    def forward(self, logits):
        return logits / self.temperature

# Focal Loss for handling imbalanced datasets
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, outputs, targets):
        BCE_loss = F.cross_entropy(outputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)  # Get the probability
        focal_loss = self.alpha * (1-pt)**self.gamma * BCE_loss
        return focal_loss.mean()

# Custom Loss Function with Focal Loss and Penalty for Wrong Predictions after 50% Accuracy
def custom_loss_function(outputs, targets, current_accuracy):
    # Apply softmax to get probabilities
    probabilities = F.softmax(outputs, dim=1)
    
    # Get the max probability (confidence) and corresponding predicted class
    confidences, predicted_classes = torch.max(probabilities, dim=1)
    
    # Calculate the Focal Loss for class imbalance
    focal_loss = FocalLoss()(outputs, targets)
    
    # Heavily penalize wrong argmax predictions if training accuracy > 50%
    wrong_predictions = (predicted_classes != targets).float()
    if current_accuracy > 0.5:
        wrong_prediction_penalty = PENALTY_WEIGHT * wrong_predictions.sum()
    else:
        wrong_prediction_penalty = 0

    # Calculate the total loss
    total_loss = focal_loss + wrong_prediction_penalty
    return total_loss

# WideResNeXt Block
class WideResNeXtBlock(nn.Module):
    expansion = 2  # Expansion factor for WideResNeXt

    def __init__(self, in_planes, planes, stride=1, cardinality=32, widen_factor=2):
        super(WideResNeXtBlock, self).__init__()
        D = cardinality * widen_factor
        self.conv1 = nn.Conv2d(in_planes, D, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(D)
        self.conv2 = nn.Conv2d(D, D, kernel_size=3, stride=stride, padding=1, groups=cardinality, bias=False)
        self.bn2 = nn.BatchNorm2d(D)
        self.conv3 = nn.Conv2d(D, planes * WideResNeXtBlock.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * WideResNeXtBlock.expansion)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes * WideResNeXtBlock.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes * WideResNeXtBlock.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * WideResNeXtBlock.expansion)
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = torch.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = torch.relu(out)
        return out

# WideResNeXt Model
class WideResNeXt(nn.Module):
    def __init__(self, block, num_blocks, cardinality=32, widen_factor=2, num_classes=100):
        super(WideResNeXt, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)

        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1, cardinality=cardinality, widen_factor=widen_factor)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2, cardinality=cardinality, widen_factor=widen_factor)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2, cardinality=cardinality, widen_factor=widen_factor)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2, cardinality=cardinality, widen_factor=widen_factor)

        self.linear = nn.Linear(512 * WideResNeXtBlock.expansion, num_classes)
        self.temperature_scaling = TemperatureScaling()  # Add temperature scaling

    def _make_layer(self, block, planes, num_blocks, stride, cardinality, widen_factor):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride, cardinality, widen_factor))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = torch.nn.functional.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        out = self.temperature_scaling(out)  # Apply temperature scaling to logits
        return out

# WideResNeXt-101 Model Definition
def WideResNeXt101():
    return WideResNeXt(WideResNeXtBlock, [3, 4, 23, 3], cardinality=32, widen_factor=2)

# Custom Dataset class for loading CIFAR100 data from .pkl files
class CIFAR100Dataset(Dataset):
    def __init__(self, file_path):
        with open(file_path, 'rb') as f:
            self.data = pickle.load(f)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image, label = self.data[idx]
        return image, label

# Training function with temperature scaling
def train_with_penalty(epoch, class_thresholds):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        
        # Calculate the overall training accuracy before updating weights
        probabilities = F.softmax(outputs, dim=1)
        _, predicted_classes = torch.max(probabilities, dim=1)
        correct_predictions = predicted_classes.eq(targets).sum().item()
        total += targets.size(0)
        current_accuracy = correct_predictions / total
        
        # Calculate custom loss with penalties if training accuracy is above 50%
        loss = custom_loss_function(outputs, targets, current_accuracy)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        correct += correct_predictions

        if batch_idx % 100 == 0:
            print(f'Epoch {epoch}, Batch {batch_idx}, Loss: {train_loss / (batch_idx + 1):.3f}, Acc: {100.*correct/total:.3f}%')


# Test function with class-specific thresholds and evaluation
def test_with_class_thresholds(epoch, class_thresholds, test_info_path):
    global best_accuracy
    model.eval()
    correct = 0
    total = 0
    high_conf_correct = 0
    high_conf_total = 0
    
    predictions = []  # Store predictions for submission
    ids = []  # Store corresponding IDs for submission
    
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            probabilities = F.softmax(outputs, dim=1)
            confidences, predicted_classes = torch.max(probabilities, dim=1)
            
            # Count correct predictions
            correct_predictions = predicted_classes.eq(targets).sum().item()
            correct += correct_predictions
            total += targets.size(0)
            
            # Class-specific thresholds: Store -1 if confidence is below class-specific threshold
            for i in range(len(inputs)):
                class_idx = predicted_classes[i].item()
                confidence = confidences[i].item()  # Convert to a float for comparison
                
                if confidence >= class_thresholds[class_idx]:  # Use >= to ensure classes at threshold are included
                    predictions.append(class_idx)  # Save the predicted class
                else:
                    predictions.append(-1)  # Assign -1 for low-confidence predictions
                
                ids.append(batch_idx * testloader.batch_size + i)  # Ensure the ID is calculated correctly

    # Save predictions to CSV
    save_predictions_to_csv(ids, predictions)
    
    # Evaluate using calculate_score
    evaluate_and_print_score(test_info_path, 'submission.csv')

    # Calculate accuracy
    test_accuracy = 100. * correct / total
    print(f"Test Accuracy: {test_accuracy:.2f}%")

# Function to save predictions to CSV in correct format
def save_predictions_to_csv(ids, predictions):
    df = pd.DataFrame({'ID': ids, 'Predicted_label': predictions})
    df.to_csv('submission.csv', index=False)
    print("Predictions saved to submission.csv")



# Function to evaluate and print score for each epoch
def evaluate_and_print_score(test_info_path, prediction_file):
    batch_1, batch_2 = evaluate_in_two_sets(test_info_path, prediction_file)
    
    # Print scores for both sets
    print(f"\nBatch 1: Correct: {batch_1['correct']}, Incorrect: {batch_1['incorrect']}, Skipped: {batch_1['skipped']}, Score: {batch_1['score']:.2f}")
    print(f"Batch 2: Correct: {batch_2['correct']}, Incorrect: {batch_2['incorrect']}, Skipped: {batch_2['skipped']}, Score: {batch_2['score']:.2f}")

# Function to calculate the score for predictions
# Function to calculate the score for predictions
def calculate_score(true_labels: pd.Series, predicted_labels: pd.Series, accuracy_threshold: float = 0.7, gamma: float = 5.0):
    data = pd.DataFrame({'True_label': true_labels, 'Predicted_label': predicted_labels})
    
    # Count skipped predictions where Predicted_label is -1
    total_skipped = (data['Predicted_label'] == -1).sum()

    # Filter out skipped predictions for further accuracy calculations
    filtered_df = data[data['Predicted_label'] != -1]
    
    all_classes = list(range(100))  # Assuming CIFAR-100 dataset with 100 classes
    sum_of_correctly_classified_high_accuracy = 0
    sum_of_correctly_classified_low_accuracy = 0
    total_correct = 0
    total_incorrect = 0
    
    accuracy_per_class = {}
    grouped = filtered_df.groupby('Predicted_label')
    
    for name, group in grouped:
        accuracy = (group['True_label'] == group['Predicted_label']).sum() / len(group)
        accuracy_per_class[name] = accuracy
        total_correct += (group['True_label'] == group['Predicted_label']).sum()
        total_incorrect += (group['True_label'] != group['Predicted_label']).sum()
    
    for cls in all_classes:
        total = len(filtered_df[filtered_df['Predicted_label'] == cls])
        class_accuracy = accuracy_per_class.get(cls, 0.0)
        
        if class_accuracy >= accuracy_threshold:
            sum_of_correctly_classified_high_accuracy += total
        else:
            sum_of_correctly_classified_low_accuracy += total

    final_score = sum_of_correctly_classified_high_accuracy - gamma * sum_of_correctly_classified_low_accuracy
    return final_score, total_correct, total_incorrect, total_skipped


# Function to evaluate the test predictions in two sets
def evaluate_in_two_sets(test_info_path: str, prediction_file: str):
    test_info = pd.read_csv(test_info_path)
    predictions = pd.read_csv(prediction_file)
    merged_data = pd.merge(test_info, predictions, on='ID')
    
    first_half = merged_data[:10000]
    second_half = merged_data[10000:20000]
    
    score_first_half, correct_first_half, incorrect_first_half, skipped_first_half = calculate_score(first_half['True_label'], first_half['Predicted_label'])
    score_second_half, correct_second_half, incorrect_second_half, skipped_second_half = calculate_score(second_half['True_label'], second_half['Predicted_label'])
    
    batch_1 = {
        'total': 10000,
        'correct': correct_first_half,
        'incorrect': incorrect_first_half,
        'skipped': skipped_first_half,
        'score': score_first_half
    }
    
    batch_2 = {
        'total': 10000,
        'correct': correct_second_half,
        'incorrect': incorrect_second_half,
        'skipped': skipped_second_half,
        'score': score_second_half
    }
    
    return batch_1, batch_2

# Generate class-specific thresholds
def generate_class_thresholds(trainloader, target_accuracy=99.99):
    class_confidences = {i: [] for i in range(100)}  # Store confidences per class

    model.eval()
    with torch.no_grad():
        for inputs, targets in trainloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            probabilities = F.softmax(outputs, dim=1)
            confidences, predicted_classes = torch.max(probabilities, dim=1)
            
            for i in range(len(targets)):
                class_idx = targets[i].item()
                class_confidences[class_idx].append(confidences[i].item())

    class_thresholds = {}
    for cls, confidences in class_confidences.items():
        sorted_confidences = sorted(confidences)
        threshold_idx = int(len(sorted_confidences) * (1 - target_accuracy / 100))
        class_thresholds[cls] = sorted_confidences[threshold_idx]
    
    return class_thresholds

# Load datasets
train_dataset = CIFAR100Dataset('train.pkl')
test_dataset = CIFAR100Dataset('test.pkl')

trainloader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
testloader = DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=2)

# Model, loss, optimizer, and scheduler
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = WideResNeXt101().to(device)

optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = StepLR(optimizer, step_size=30, gamma=0.1)

best_accuracy = 0.0

# Generate class-specific thresholds based on 99.99% accuracy
class_thresholds = generate_class_thresholds(trainloader, target_accuracy=80)

# Example Training Loop
for epoch in range(0, 100):
    train_with_penalty(epoch, class_thresholds)
    test_with_class_thresholds(epoch, class_thresholds, 'test_info.csv')
    scheduler.step()


/opt/conda/lib/python3.10/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


Epoch 0, Batch 0, Loss: 4.581, Acc: 2.344%
Epoch 0, Batch 100, Loss: 4.522, Acc: 1.825%
Epoch 0, Batch 200, Loss: 4.329, Acc: 2.791%
Epoch 0, Batch 300, Loss: 4.164, Acc: 4.036%
Predictions saved to submission.csv

Batch 1: Correct: 767, Incorrect: 9233, Skipped: 0, Score: -50000.00
Batch 2: Correct: 740, Incorrect: 9260, Skipped: 0, Score: -50000.00
Test Accuracy: 0.01%
Epoch 1, Batch 0, Loss: 3.536, Acc: 10.156%
Epoch 1, Batch 100, Loss: 3.617, Acc: 9.251%
Epoch 1, Batch 200, Loss: 3.526, Acc: 10.599%
Epoch 1, Batch 300, Loss: 3.449, Acc: 11.786%
Predictions saved to submission.csv

Batch 1: Correct: 1386, Incorrect: 8614, Skipped: 0, Score: -49856.00
Batch 2: Correct: 1428, Incorrect: 8572, Skipped: 0, Score: -49604.00
Test Accuracy: 0.01%
Epoch 2, Batch 0, Loss: 3.205, Acc: 12.500%
Epoch 2, Batch 100, Loss: 3.073, Acc: 17.814%
Epoch 2, Batch 200, Loss: 3.032, Acc: 18.315%
Epoch 2, Batch 300, Loss: 2.979, Acc: 19.295%
Predictions saved to submission.csv

Batch 1: Correct: 2006, Inco